In [2]:
!pip install netCDF4

  Obtaining dependency information for netCDF4 from https://files.pythonhosted.org/packages/b6/66/6fcd815bf53b3e1653a02356e84ed23d191baaa844c5caad31d20a31cce8/netCDF4-1.6.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cftime from https://files.pythonhosted.org/packages/a8/d2/75554106ea5bd1d44c73d354155395846d3833a5effe3a8660adde385288/cftime-1.6.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/6.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/6.6 MB 375.8 kB/s eta 0:00:18
   ---------------------------------------- 0.1/6.6 MB 563.7 kB/s eta 0:00:12
    --------------------------------------- 0.1/6.6 MB 654.9 kB/s eta 0:00:10
   - -------------------------------------- 0.2/6.6 MB 696.3 kB/s eta 0:00:10
   - -------------------------------------- 0.2/6.6 MB 769.9 kB/s e

In [4]:
!pip install basemap basemap-data-hires

  Obtaining dependency information for basemap from https://files.pythonhosted.org/packages/36/91/c3bf3ef6eacdb7cae204f3b440b7810ecbb30a4067457310d2544c6a5b14/basemap-1.4.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for basemap-data-hires from https://files.pythonhosted.org/packages/11/3a/4a975fc74b63e2d494e236ba70fe765151bf0adad758ad40e4cf4ecf2310/basemap_data_hires-1.3.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for basemap-data<1.4,>=1.3.2 from https://files.pythonhosted.org/packages/2c/c3/63fdb885308c999206c7a497d79a50891581626da7dc491d432d2732bb46/basemap_data-1.3.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyshp<2.4,>=1.2 from https://files.pythonhosted.org/packages/98/2f/68116db5b36b895c0450e3072b8cb6c2fac0359279b182ea97014d3c8ac0/pyshp-2.3.1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ------- -------------------------------- 10.2/56.0 kB ? 

In [5]:
from PIL import Image, ImageSequence
import pandas as pd 
import glob
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["PROJ_LIB"] = "C:\\Users\\ishan\\anaconda3\\Library\\share"; #fixr
from mpl_toolkits.basemap import Basemap

In [9]:
import os
import glob
from PIL import Image
import numpy as np
from netCDF4 import Dataset

def convert_dms_to_dec(degrees, minutes, seconds, north=True, east=True):
  """
  Converts degrees, minutes, and seconds to decimal degrees.

  Args:
      degrees: Degrees (int).
      minutes: Minutes (int).
      seconds: Seconds (float).
      north: True if north latitude, False if south latitude (default: True).
      east: True if east longitude, False if west longitude (default: True).

  Returns:
      Decimal representation of the DMS coordinate.
  """

  decimal_degrees = float(degrees) + float(minutes) / 60 + seconds / 3600
  if not north:
    decimal_degrees = -decimal_degrees
  if not east:
    decimal_degrees = -decimal_degrees
  return decimal_degrees


# Define the directory containing the images
image_directory = "D:\\sem6\\hackathons\\cmrit\\mapdata"
output_directory = 'D:\\sem6\\hackathons\\cmrit\\ncout2'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Find all image files in the directory 
image_files = glob.glob(os.path.join(image_directory, '*.jpg'))

# Iterate through each image file
for image_file in image_files:
    # Open the image
    img = Image.open(image_file)
    
    # Convert the image to a numpy array
    data = np.array(img)
    
    # Define corner coordinates in decimal degrees
    lat_min = convert_dms_to_dec(12, 52, 0, north=True)  # Assuming north
    lat_max = convert_dms_to_dec(13, 4, 0, north=True)  # Assuming north
    lon_min = convert_dms_to_dec(77, 32, 0, east=True)  # Assuming east
    lon_max = convert_dms_to_dec(77, 44, 0, east=True)  # Assuming east

    # Create linspace ranges for latitudes and longitudes
    latitudes = np.linspace(lat_min, lat_max, data.shape[0])
    longitudes = np.linspace(lon_min, lon_max, data.shape[1])

    # Create the NetCDF file
    output_file = os.path.join(output_directory, os.path.basename(image_file).replace('.jpg', '.nc'))
    nc_file = Dataset(output_file, 'w', format='NETCDF4')
    
    # Create dimensions
    lat_dim = nc_file.createDimension('latitude', len(latitudes))
    lon_dim = nc_file.createDimension('longitude', len(longitudes))
    
    # Create variables
    latitudes_var = nc_file.createVariable('latitude', np.float32, ('latitude',))
    longitudes_var = nc_file.createVariable('longitude', np.float32, ('longitude',))
    data_var = nc_file.createVariable('data', np.float32, ('latitude', 'longitude'))
    
    # Assign data to variables
    latitudes_var[:] = latitudes
    longitudes_var[:] = longitudes
    data_var[:, :,] = data
    
    # Add attributes
    latitudes_var.units = 'degrees_north'
    longitudes_var.units = 'degrees_east'
    data_var.units = 'unknown'  # Update this based on your data
    
    # Close the NetCDF file
    nc_file.close()
    
    print(f'NetCDF file created for {image_file}: {output_file}')

print('Conversion complete.')


ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (1280, 1280) and arg 1 with shape (1280, 1280, 3).